In [1]:
import torch
from transformers import pipeline
device = torch.device('cuda')

classifier = pipeline('sentiment-analysis',model="nlptown/bert-base-multilingual-uncased-sentiment")



/home/dheeraj/Development/Pytorch Learning/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


In [2]:
classifier("We are very happy to show you the transformer library")

[{'label': '5 stars', 'score': 0.683954119682312}]

In [16]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification

In [17]:
classifier('I am a good boy')

[{'label': '4 stars', 'score': 0.4229268729686737}]

In [ ]:
model_name = 'distilbert-base-uncased-finetuned'
tokenizer = AutoTokenizer.from_pretrained(model_name)

NameError: name 'model_name' is not defined